# Compute LD and PCA via PLINK

In this notebook, we compute linkage disequilibrium and principal components analysis using [PLINK2](https://www.cog-genomics.org/plink/2.0/).

Note that this work is part of a larger project to [Demonstrate the Potential for Pooled Analysis of All of Us and UK Biobank Genomic Data](https://docs.google.com/document/d/19ZS0z_-7FEM37pNDAXaWaqBSLnqyd9MZEkiOmtF3n_0/edit#). Specifically this is for the portion of the project that is the **pooled** analysis.

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the <i>All of Us</i> Workbench.
    <ul>
        <li>Use "Recommended Environment" <kbd><b>General Analysis</b></kbd> which creates compute type <kbd><b>Standard VM</b></kbd> with sufficient CPU and RAM (e.g. start with <b>52 CPUs</b> and <b>30 GB RAM</b>, increase if needed).</li>
        <li>This notebook can take a while to run <b>TBD DETAILS CHR21 VS. ALL CHRS</b>. Recommend that it is run in the background via <kbd>run_notebook_in_the_background</kbd>.</li>    </ul>
</div>

In [ ]:
from datetime import datetime
import os
import time

## Setup plink2

https://www.cog-genomics.org/plink/2.0/

In [ ]:
%%bash

##### plink 2 install
PLINK_VERSION=2.3.Alpha
PLINK_ZIP_PATH=/tmp/plink-$PLINK_VERSION.zip
curl -L -o $PLINK_ZIP_PATH https://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
mkdir -p /tmp/plink2/
unzip -o $PLINK_ZIP_PATH -d /tmp/plink2/

In [ ]:
!/tmp/plink2/plink2 --version # --help

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# The BGEN file created via aou_workbench_pooled_analyses/05_write_pooled_bgen.ipynb.
REMOTE_MERGED_BGEN = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/geno/20220215/aou-alpha3-ukb-chr1-chr22.bgen'
# The sample file created via aou_workbench_pooled_analyses/05_write_pooled_bgen.ipynb.
REMOTE_MERGED_BGEN_SAMPLE = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/geno/20220215/aou-alpha3-ukb-chr1-chr22.sample'
# These two files were created via notebook aou_workbench_pooled_analyses/06_pooled_variant_qc.ipynb
# NOTE: use variant QC files created for regenie step 2, not step 1, here.
REMOTE_VARIANT_QC_ID = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/variant-qc/20220215/aou_alpha3_ukb_lipids_step2QC_plink.id'
REMOTE_VARIANT_QC_SNPLIST = 'gs://fc-secure-e53e4a44-7fe2-42b7-89b7-01aae1e399f7/data/pooled/variant-qc/20220215/aou_alpha3_ukb_lipids_step2QC_plink.snplist'

#---[ Outputs ]---
# Create a timestamp for a folder of results generated today.
DATESTAMP = time.strftime('%Y%m%d')
OUTPUT_FILENAME_PREFIX = 'aou_alpha3_ukb_lipids'
OUTPUT_FOLDER = f'{os.getenv("WORKSPACE_BUCKET")}/data/pooled/ld-pca/{DATESTAMP}/'

In [ ]:
LOCAL_MERGED_BGEN = os.path.basename(REMOTE_MERGED_BGEN)
LOCAL_MERGED_BGEN_SAMPLE = os.path.basename(REMOTE_MERGED_BGEN_SAMPLE)
LOCAL_VARIANT_QC_ID = os.path.basename(REMOTE_VARIANT_QC_ID)
LOCAL_VARIANT_QC_SNPLIST = os.path.basename(REMOTE_VARIANT_QC_SNPLIST)

## Copy data locally

In [ ]:
!gsutil cp -n {REMOTE_MERGED_BGEN} {REMOTE_MERGED_BGEN_SAMPLE} .

In [ ]:
!gsutil cp {REMOTE_VARIANT_QC_ID} {REMOTE_VARIANT_QC_SNPLIST} .

# Compute linkage disequilibrium via plink2

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_MERGED_BGEN} ref-first \
  --sample {LOCAL_MERGED_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_VARIANT_QC_ID} \
  --extract {LOCAL_VARIANT_QC_SNPLIST} \
  --indep-pairwise 200 50 0.25 \
  --out {OUTPUT_FILENAME_PREFIX}_plink_ld

In [ ]:
%%bash

ls -lat | head

In [ ]:
%%bash

wc -l *prune*

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}_plink_ld* {OUTPUT_FOLDER}

In [ ]:
!gsutil ls {OUTPUT_FOLDER}

# Compute principal components analysis via plink2

<div class="alert alert-block alert-warning">
    <b>Note</b>: the <kbd>--memory</kbd> parameter below assumes the machine has 52 GB of RAM. Adjust this value if the machine has more or less than 52 GB of RAM.
</div>

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_MERGED_BGEN} ref-first \
  --sample {LOCAL_MERGED_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_VARIANT_QC_ID} \
  --extract {OUTPUT_FILENAME_PREFIX}_plink_ld.prune.in \
  --pca 15 approx \
  --memory 50000 \
  --out {OUTPUT_FILENAME_PREFIX}_plink_pca

In [ ]:
%%bash

ls -lat | head

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}_plink_pca* {OUTPUT_FOLDER}

In [ ]:
!gsutil ls {OUTPUT_FOLDER}

# Provenance 

In [ ]:
%%bash

date

In [ ]:
%%bash

pip3 freeze